In [4]:
%pip install tensorflow-metal

  Using cached tensorflow_metal-1.1.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (1.2 kB)
Using cached tensorflow_metal-1.1.0-cp311-cp311-macosx_12_0_arm64.whl (1.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:

"""
This script has the method
preprocess_data(X, Y):
"""
import tensorflow.keras as K

#import keras as K


def preprocess_data(X, Y):
        """ This method has the preprocess to train a model """
        X = X.astype('float32')
        X_p = K.applications.vgg16.preprocess_input(X)
        Y_p = K.utils.to_categorical(Y, 10)
        return(X_p, Y_p)

if __name__ == "__main__":
    (Xt, Yt), (X, Y) = K.datasets.cifar10.load_data()
    X_p, Y_p = preprocess_data(Xt, Yt)
    Xv_p, Yv_p = preprocess_data(X, Y)
    base_model = K.applications.vgg16.VGG16(include_top=False,
                                            weights='imagenet',
                                            pooling='avg'
                                            )

    model= K.Sequential()
    model.add(K.layers.UpSampling2D())
    model.add(base_model)
    model.add(K.layers.Flatten())
    model.add(K.layers.Dense(512, activation=('relu'))) 
    model.add(K.layers.Dropout(0.2))
    model.add(K.layers.Dense(256, activation=('relu')))
    model.add(K.layers.Dropout(0.2))
    model.add(K.layers.Dense(10, activation=('softmax')))
    callback = []
    
    def decay(epoch):
        """ This method create the alpha"""
        return 0.001 / (1 + 1 * 30)
    callback += [K.callbacks.LearningRateScheduler(decay, verbose=1)]
    callback += [K.callbacks.ModelCheckpoint('cifar10.h5',
                                             save_best_only=True,
                                             mode='min'
                                             )]
    model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(x=X_p, y=Y_p,
              batch_size=128,
              validation_data=(Xv_p, Yv_p),
              epochs=10, shuffle=True,
              callbacks=callback,
              verbose=1
              )


Epoch 1: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 1/10


2024-03-19 19:21:20.214782: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - ETA: 0s - loss: 1.8882 - accuracy: 0.5075

2024-03-19 19:23:15.227798: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - 122s 309ms/step - loss: 1.8882 - accuracy: 0.5075 - val_loss: 0.8141 - val_accuracy: 0.7766 - lr: 3.2258e-05

Epoch 2: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 2/10
391/391 [==============================] - 123s 315ms/step - loss: 0.8413 - accuracy: 0.7859 - val_loss: 0.5809 - val_accuracy: 0.8505 - lr: 3.2258e-05

Epoch 3: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 3/10
391/391 [==============================] - 123s 315ms/step - loss: 0.6416 - accuracy: 0.8437 - val_loss: 0.4949 - val_accuracy: 0.8821 - lr: 3.2258e-05

Epoch 4: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 4/10
391/391 [==============================] - 123s 315ms/step - loss: 0.7113 - accuracy: 0.8660 - val_loss: 0.9719 - val_accuracy: 0.8277 - lr: 3.2258e-05

Epoch 5: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 5/10
391/391 [===============

In [4]:
from keras.models import load_model

model = load_model('cifar10.h5')

In [5]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

predictions = model.predict(Xv_p)
predicted_labels = np.argmax(predictions, axis=1)

# Calculating metrics
print("Confusion Matrix:")
conf_matrix = confusion_matrix(np.argmax(Yv_p, axis=1), predicted_labels)
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(np.argmax(Yv_p, axis=1), predicted_labels))


  4/313 [..............................] - ETA: 7s 

2024-03-19 19:41:48.044096: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 7s 22ms/step
Confusion Matrix:
[[911   2  13   3   4   0   6  16  24  21]
 [ 10 905   0   0   0   0   3   1  10  71]
 [ 37   0 844  21  32  23  23  15   5   0]
 [  7   3  30 711  25 162  31  20   5   6]
 [  8   1  24  15 867  23  29  30   3   0]
 [  1   0   9  88  18 848   7  27   0   2]
 [  6   0  16  32   6  10 921   6   1   2]
 [  6   0   7   8  25  37   1 912   2   2]
 [ 27   3   5   5   0   1   3   2 947   7]
 [ 12  13   0   2   0   2   0   2  14 955]]

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.91      0.90      1000
           1       0.98      0.91      0.94      1000
           2       0.89      0.84      0.87      1000
           3       0.80      0.71      0.75      1000
           4       0.89      0.87      0.88      1000
           5       0.77      0.85      0.81      1000
           6       0.90      0.92      0.91      1000
           7       0.88      0.91     